### Distress Integration
* Extraction of keycode in excel from mdb's

In [9]:
import pyodbc,os,pandas as pd

mdb_directory = r"D:\_2.Programming\_6.test\distress_mdb" # input directory
o_keycode = r"D:\_2.Programming\_6.test\Ex_keycode.xlsx" # output directory

#defining function to read mdb's
def read_mdb(x):
    mdb_files = []
    for root, dirs, files in os.walk(x):
        for file in files:
            if file.endswith(".mdb"):
                mdb_files.append(os.path.join(root, file))
    return mdb_files

#defining function to read tables in mdb
def read_tables_by_prefix(mdb_file, table_prefix):
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={mdb_file};"
    connection = pyodbc.connect(conn_str)
    cursor = connection.cursor()

    tables = []
    for table_info in cursor.tables(tableType='TABLE'):
        table_name = table_info.table_name
        if table_name.startswith(table_prefix):
            tables.append(table_name)

    data = pd.DataFrame()
    for table in tables:
        query = f"SELECT * FROM [{table}]"
        table_data = pd.read_sql(query, connection)
        table_data['Survey_ID'] = os.path.splitext(os.path.basename(mdb_file))[0]
        data = pd.concat([data, table_data], ignore_index=True)

    connection.close()
    return data

#calling function to read mdb's
mdb_files = read_mdb(mdb_directory)

#pandas data frame for storing extracted keycode
Key_code = pd.DataFrame()

#extracting and merging keycode from mdb's 
for mdb_file in mdb_files:
    keycd = read_tables_by_prefix(mdb_file, "dt_VideoKeyCode_Raw")
    Key_code = pd.concat([Key_code, keycd], ignore_index=True)

#exporting  keycode from pandas data frame
Key_code.to_excel(o_keycode, index=False)

print(len(mdb_files))
print("Successfully completed!!")


C:\Users\asifi\AppData\Local\Temp\ipykernel_33380\3757654658.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_33380\3757654658.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_33380\3757654658.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_33380\37576

57
Successfully completed!!


### Creating distress feature classes from excel sheet
* based on chainage, creating two distress feature class i.e Point and line distress

In [ ]:
o_keycode = r"D:\_2.Programming\_6.test\Ex_keycode.xlsx" 
arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

DV_keycode = pd.read_excel(o_keycode)
field = 'EVENT_DESC'
field_values = ['REMARKS','PAVEMENT MARKING']
distress = DV_keycode[~DV_keycode[field].isin(field_values)]
distress_Remark = DV_keycode[DV_keycode[field].isin(field_values)]

#exporting distress and remarks excel from pandas data frame
distress.to_excel(o_keycode, index = False)
distress_Remark.to_excel(o_keycode, index = False) 

#converting distress and remarks excel workbooks into tables
arcpy.ExcelToTable_conversion(o_keycode, "Keycode_tbl", "Sheet1")
arcpy.ExcelToTable_conversion(o_keycode, "Remarks_tbl", "Sheet1")

#creating point distress by select by attribute method, where chainage start and chainage end are equal
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "NEW_SELECTION", '"CHAINAGE_START" = "CHAINAGE_END" ')
arcpy.TableToTable_conversion("Keycode_tbl", arcpy.env.workspace, "P_Distress")
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "CLEAR_SELECTION")

#creating Line distress by select by attribute method, where chainage start and end is not equal
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "NEW_SELECTION", '"CHAINAGE_START" <> "CHAINAGE_END" ')
arcpy.TableToTable_conversion("Keycode_tbl", arcpy.env.workspace, "L_Distress")
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "CLEAR_SELECTION")


#creating feature point and line classes from point and line distess tables 
arcpy.management.XYTableToPoint("P_Distress","Distress_P_wgs","X","Y","",arcpy.SpatialReference(4326))
arcpy.management.XYToLine("L_Distress","Distress_L_wgs","X","Y","X_END","Y_END","GEODESIC","",arcpy.SpatialReference(4326),True)

utm_crs = arcpy.SpatialReference(32640)
arcpy.Project_management("Distress_P_wgs","P_distress_utm",utm_crs)
arcpy.Project_management("Distress_L_wgs","L_distress_utm",utm_crs)

print("Successfully completed!!")

### The below script will follow after snapping of distress line to pms sections
* first, start and end vertexs of pms section will produced which used for splitting line distresses
* lines distress will under go for splitting based on points produced from pms sections at start and end vertexes
* Quantity value will be calculated

In [ ]:
from collections import Counter

arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

arcpy.management.AddFields("L_distress_utm",[["section_occurence","TEXT","",300],["split_len","TEXT"],["f_part","TEXT"],
                                               ["s_part","TEXT"],["qc","TEXT"],["Quantity","DOUBLE"],["Section_len","DOUBLE"]])

arcpy.management.AddFields("P_distress_utm",[["section_occurence","TEXT","",300],["split_len","TEXT"],[ "f_part","TEXT"],
                                               ["s_part","TEXT"],["qc","TEXT"],["Quantity","DOUBLE"],["Section_len","DOUBLE"]])    


arcpy.management.CalculateField("L_distress_utm","Section_len", "!SHAPE.LENGTH!", "PYTHON3")

expr = f"!SURVEY_ID!+!COMMENT_!+!COMMENT_1!+!EVENT_DESC!+str(!CHAINAGE_START!)+str(!CHAINAGE_END!)+str(!LENGTH!)"
 
arcpy.management.CalculateField("L_distress_utm","section_occurence", expr, "PYTHON3")

expr1 = f"!SURVEY_ID!+!COMMENT_1!+!EVENT_DESC!+str(!CHAINAGE_START!)+str(!CHAINAGE_END!)+str(!LENGTH!)"
arcpy.management.CalculateField("P_distress_utm","section_occurence", expr1, "PYTHON3")

arcpy.management.FeatureVerticesToPoints("PMS_Sections","PMS_sec_Points","BOTH_ENDS")
arcpy.management.SplitLineAtPoint("L_distress_utm","L_GPS_Points", "Distress_L_split","0.5 Meters")



def multiply_fields(a, b, c, d):
    try:
        a_value = float(a) if a is not None else 1.0
        b_value = float(b) if b is not None else 1.0
        c_value = float(c) if c is not None else 1.0
        d_value = float(d) if d is not None else 1.0
        result = a_value * b_value * c_value * d_value
        return result
    except (ValueError,TypeError):
        return None
    

def multiply_p_distress(a, b, c):
    try:
        a_value = float(a) if a is not None else 1.0
        b_value = float(b) if b is not None else 1.0
        c_value = float(c) if c is not None else 1.0
        result = a_value * b_value * c_value
        return result
    except (ValueError,TypeError):
        return None
    
# Use a Search Cursor to count occurrences of each distress_L
section_counter = Counter()
with arcpy.da.SearchCursor("Distress_L_split", ["section_occurence"]) as cursor:
    for row in cursor:
        section_counter[row[0]] += 1


with arcpy.da.UpdateCursor("Distress_L_split", ["section_occurence", 'SHAPE@', "COMMENT_1", "split_len", "f_part",
                                       "s_part","qc","Quantity","Section_len"]) as cursor:
    for row in cursor:
        same_sections = row[0]
        if section_counter[same_sections] > 1 and '*' not in row[2]:      # distress length fall more than one section for linear distress
            row[7] = (((row[1].length)/row[8]) * (float(row[2])))         # fallen length/distressL * ShapeLength
        elif section_counter[same_sections] > 1 and '*' in row[2]:        # distress length fall in more than one section for area distress
            row[4] = row[2].split("*")[-1]                                # getting width of distress
            row[7] = float(row[4]) * row[1].length                        # splited distress lenght multiply by width
        elif section_counter[same_sections] == 1 and '*' not in row[2]:   # distress length fall in one section for linear distress
            row[7] = float(row[2])
        elif section_counter[same_sections] == 1 and '*' in row[2]:       # distress length fall in one section for area distress
            row[4] = row[2].split("*")[0]
            row[5] = row[2].split("*")[-1]
            row[7] = float(row[4]) * float(row[5])  
        else:
            row[6] = "check the values"
        cursor.updateRow(row)

with arcpy.da.UpdateCursor("P_distress_utm", ["COMMENT_1", "split_len", "f_part", "qc", "Quantity", "Section_len"]) as cursor:
    for row in cursor:
        if "*" in row[0]:
            row[1] = row[0].split("*")[0]
            row[2] = row[0].split("*")[-1]
            row[4] = float(row[1]) * float(row[2])
        elif "*" not in row[0]:
            row[4] = float(row[0])
        else:
            row[3] = "check the values"
        cursor.updateRow(row)
     
print ("sucessfully completed")